In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
    
batch_size = 128
model_name = "resnet20_quant4bit"
model = resnet20_quant()
#model.conv1 = nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1, bias=False)
#model.bn1 = bn1 = nn.BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
model.layer1[0].conv1 = QuantConv2d(8, 8, kernel_size=3, stride=1, padding=1, bias=False)
#model.layer1[0].conv2 = QuantConv2d(8, 8, kernel_size=3, stride=1, padding=1, bias=False)
#model.layer1[0].bn1 = nn.BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
model.layer1[0].bn2 = nn.Sequential()
#model.layer1[1].conv1 = QuantConv2d(8, 16, kernel_size=3, stride=1, padding=1, bias=False)
print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [80, 120]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
ResNet_Cifar(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantConv2d(
        8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (conv2): QuantConv2d(
        8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (bn2): Sequential()
    )
    (1): BasicBlock(
      (conv1): QuantConv2d(
        8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (conv2): QuantConv2d(
        8, 8, kernel_size=(3, 3), strid

In [2]:
# This cell won't be given, but students will complete the training

lr = 4e-2
weight_decay = 1e-4
epochs = 100
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

Epoch: [0][0/391]	Time 0.450 (0.450)	Data 0.264 (0.264)	Loss 2.3541 (2.3541)	Prec 11.719% (11.719%)
Epoch: [0][100/391]	Time 0.085 (0.085)	Data 0.002 (0.005)	Loss 1.8977 (2.0458)	Prec 28.125% (21.968%)
Epoch: [0][200/391]	Time 0.079 (0.078)	Data 0.002 (0.004)	Loss 1.6753 (1.8851)	Prec 31.250% (28.148%)
Epoch: [0][300/391]	Time 0.087 (0.080)	Data 0.002 (0.003)	Loss 1.5059 (1.7835)	Prec 43.750% (32.345%)
Validation starts
Test: [0/79]	Time 0.243 (0.243)	Loss 1.4584 (1.4584)	Prec 47.656% (47.656%)
 * Prec 46.960% 
best acc: 46.960000
Epoch: [1][0/391]	Time 0.273 (0.273)	Data 0.227 (0.227)	Loss 1.3085 (1.3085)	Prec 51.562% (51.562%)
Epoch: [1][100/391]	Time 0.081 (0.086)	Data 0.002 (0.005)	Loss 1.3366 (1.3863)	Prec 49.219% (48.933%)
Epoch: [1][200/391]	Time 0.075 (0.079)	Data 0.003 (0.003)	Loss 1.1047 (1.3530)	Prec 60.938% (50.412%)
Epoch: [1][300/391]	Time 0.078 (0.080)	Data 0.002 (0.003)	Loss 1.1065 (1.3178)	Prec 57.812% (51.936%)
Validation starts
Test: [0/79]	Time 0.229 (0.229)	Loss 1.

Epoch: [15][200/391]	Time 0.065 (0.085)	Data 0.002 (0.003)	Loss 0.4800 (0.5088)	Prec 84.375% (82.521%)
Epoch: [15][300/391]	Time 0.089 (0.082)	Data 0.002 (0.003)	Loss 0.4534 (0.5084)	Prec 82.812% (82.478%)
Validation starts
Test: [0/79]	Time 0.281 (0.281)	Loss 0.5133 (0.5133)	Prec 82.812% (82.812%)
 * Prec 81.150% 
best acc: 81.150000
Epoch: [16][0/391]	Time 0.294 (0.294)	Data 0.221 (0.221)	Loss 0.5418 (0.5418)	Prec 79.688% (79.688%)
Epoch: [16][100/391]	Time 0.085 (0.086)	Data 0.002 (0.004)	Loss 0.5579 (0.4868)	Prec 80.469% (82.782%)
Epoch: [16][200/391]	Time 0.087 (0.083)	Data 0.003 (0.003)	Loss 0.4721 (0.4972)	Prec 85.938% (82.774%)
Epoch: [16][300/391]	Time 0.079 (0.079)	Data 0.002 (0.003)	Loss 0.4302 (0.4969)	Prec 83.594% (82.742%)
Validation starts
Test: [0/79]	Time 0.246 (0.246)	Loss 0.7624 (0.7624)	Prec 74.219% (74.219%)
 * Prec 76.860% 
best acc: 81.150000
Epoch: [17][0/391]	Time 0.393 (0.393)	Data 0.313 (0.313)	Loss 0.6065 (0.6065)	Prec 80.469% (80.469%)
Epoch: [17][100/391]	

Epoch: [30][300/391]	Time 0.067 (0.083)	Data 0.002 (0.003)	Loss 0.3349 (0.3853)	Prec 89.062% (86.477%)
Validation starts
Test: [0/79]	Time 0.238 (0.238)	Loss 0.4027 (0.4027)	Prec 84.375% (84.375%)
 * Prec 82.760% 
best acc: 82.760000
Epoch: [31][0/391]	Time 0.293 (0.293)	Data 0.221 (0.221)	Loss 0.5037 (0.5037)	Prec 83.594% (83.594%)
Epoch: [31][100/391]	Time 0.087 (0.082)	Data 0.002 (0.004)	Loss 0.5843 (0.3700)	Prec 78.125% (86.935%)
Epoch: [31][200/391]	Time 0.081 (0.083)	Data 0.002 (0.003)	Loss 0.4982 (0.3711)	Prec 85.156% (86.866%)
Epoch: [31][300/391]	Time 0.083 (0.083)	Data 0.002 (0.003)	Loss 0.3629 (0.3705)	Prec 85.938% (86.846%)
Validation starts
Test: [0/79]	Time 0.226 (0.226)	Loss 0.5611 (0.5611)	Prec 80.469% (80.469%)
 * Prec 79.710% 
best acc: 82.760000
Epoch: [32][0/391]	Time 0.295 (0.295)	Data 0.236 (0.236)	Loss 0.1901 (0.1901)	Prec 93.750% (93.750%)
Epoch: [32][100/391]	Time 0.086 (0.084)	Data 0.002 (0.004)	Loss 0.2912 (0.3626)	Prec 90.625% (87.384%)
Epoch: [32][200/391]	

Validation starts
Test: [0/79]	Time 0.270 (0.270)	Loss 0.4785 (0.4785)	Prec 85.156% (85.156%)
 * Prec 84.020% 
best acc: 84.790000
Epoch: [46][0/391]	Time 0.313 (0.313)	Data 0.242 (0.242)	Loss 0.3147 (0.3147)	Prec 86.719% (86.719%)
Epoch: [46][100/391]	Time 0.087 (0.083)	Data 0.002 (0.005)	Loss 0.4413 (0.2986)	Prec 86.719% (89.341%)
Epoch: [46][200/391]	Time 0.075 (0.084)	Data 0.002 (0.003)	Loss 0.3383 (0.3008)	Prec 85.156% (89.296%)
Epoch: [46][300/391]	Time 0.084 (0.083)	Data 0.002 (0.003)	Loss 0.1763 (0.3008)	Prec 95.312% (89.351%)
Validation starts
Test: [0/79]	Time 0.266 (0.266)	Loss 0.6499 (0.6499)	Prec 83.594% (83.594%)
 * Prec 81.210% 
best acc: 84.790000
Epoch: [47][0/391]	Time 0.329 (0.329)	Data 0.267 (0.267)	Loss 0.3335 (0.3335)	Prec 89.062% (89.062%)
Epoch: [47][100/391]	Time 0.082 (0.085)	Data 0.002 (0.005)	Loss 0.1989 (0.3068)	Prec 92.969% (89.496%)
Epoch: [47][200/391]	Time 0.085 (0.085)	Data 0.002 (0.004)	Loss 0.1721 (0.3096)	Prec 92.969% (89.373%)
Epoch: [47][300/391]	

 * Prec 84.110% 
best acc: 86.230000
Epoch: [61][0/391]	Time 0.272 (0.272)	Data 0.206 (0.206)	Loss 0.2817 (0.2817)	Prec 89.062% (89.062%)
Epoch: [61][100/391]	Time 0.085 (0.073)	Data 0.002 (0.004)	Loss 0.2244 (0.2703)	Prec 92.188% (90.370%)
Epoch: [61][200/391]	Time 0.082 (0.078)	Data 0.002 (0.003)	Loss 0.2315 (0.2632)	Prec 89.062% (90.652%)
Epoch: [61][300/391]	Time 0.079 (0.079)	Data 0.002 (0.003)	Loss 0.1745 (0.2689)	Prec 92.969% (90.404%)
Validation starts
Test: [0/79]	Time 0.231 (0.231)	Loss 0.3331 (0.3331)	Prec 89.844% (89.844%)
 * Prec 85.870% 
best acc: 86.230000
Epoch: [62][0/391]	Time 0.294 (0.294)	Data 0.235 (0.235)	Loss 0.3386 (0.3386)	Prec 87.500% (87.500%)
Epoch: [62][100/391]	Time 0.081 (0.073)	Data 0.003 (0.004)	Loss 0.2238 (0.2603)	Prec 92.188% (90.888%)
Epoch: [62][200/391]	Time 0.085 (0.078)	Data 0.002 (0.003)	Loss 0.3201 (0.2649)	Prec 89.844% (90.699%)
Epoch: [62][300/391]	Time 0.087 (0.080)	Data 0.003 (0.003)	Loss 0.2967 (0.2672)	Prec 88.281% (90.586%)
Validation s

Epoch: [76][100/391]	Time 0.087 (0.087)	Data 0.003 (0.005)	Loss 0.2522 (0.2429)	Prec 89.062% (91.491%)
Epoch: [76][200/391]	Time 0.044 (0.076)	Data 0.002 (0.004)	Loss 0.3810 (0.2368)	Prec 85.938% (91.589%)
Epoch: [76][300/391]	Time 0.078 (0.070)	Data 0.003 (0.003)	Loss 0.3238 (0.2391)	Prec 89.062% (91.507%)
Validation starts
Test: [0/79]	Time 0.222 (0.222)	Loss 0.4441 (0.4441)	Prec 87.500% (87.500%)
 * Prec 85.090% 
best acc: 86.380000
Epoch: [77][0/391]	Time 0.311 (0.311)	Data 0.238 (0.238)	Loss 0.2383 (0.2383)	Prec 94.531% (94.531%)
Epoch: [77][100/391]	Time 0.067 (0.063)	Data 0.002 (0.005)	Loss 0.2346 (0.2310)	Prec 89.844% (91.901%)
Epoch: [77][200/391]	Time 0.069 (0.064)	Data 0.004 (0.004)	Loss 0.3094 (0.2367)	Prec 87.500% (91.737%)
Epoch: [77][300/391]	Time 0.064 (0.063)	Data 0.002 (0.003)	Loss 0.1871 (0.2385)	Prec 93.750% (91.609%)
Validation starts
Test: [0/79]	Time 0.220 (0.220)	Loss 0.3377 (0.3377)	Prec 89.844% (89.844%)
 * Prec 85.230% 
best acc: 86.380000
Epoch: [78][0/391]	

Epoch: [91][200/391]	Time 0.058 (0.060)	Data 0.002 (0.003)	Loss 0.1019 (0.1137)	Prec 95.312% (96.067%)
Epoch: [91][300/391]	Time 0.053 (0.059)	Data 0.002 (0.003)	Loss 0.1452 (0.1140)	Prec 96.875% (96.021%)
Validation starts
Test: [0/79]	Time 0.215 (0.215)	Loss 0.2292 (0.2292)	Prec 92.969% (92.969%)
 * Prec 89.440% 
best acc: 89.870000
Epoch: [92][0/391]	Time 0.309 (0.309)	Data 0.243 (0.243)	Loss 0.0701 (0.0701)	Prec 98.438% (98.438%)
Epoch: [92][100/391]	Time 0.066 (0.056)	Data 0.002 (0.004)	Loss 0.0794 (0.1113)	Prec 97.656% (96.272%)
Epoch: [92][200/391]	Time 0.066 (0.059)	Data 0.002 (0.003)	Loss 0.0480 (0.1160)	Prec 99.219% (96.137%)
Epoch: [92][300/391]	Time 0.056 (0.060)	Data 0.002 (0.003)	Loss 0.1624 (0.1168)	Prec 94.531% (96.070%)
Validation starts
Test: [0/79]	Time 0.232 (0.232)	Loss 0.2576 (0.2576)	Prec 89.844% (89.844%)
 * Prec 89.590% 
best acc: 89.870000
Epoch: [93][0/391]	Time 0.297 (0.297)	Data 0.227 (0.227)	Loss 0.1147 (0.1147)	Prec 94.531% (94.531%)
Epoch: [93][100/391]	

In [4]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
device = torch.device("cuda" if use_gpu else "cpu") 
for layer in model.modules():
    if isinstance(layer, torch.nn.Conv2d):
        print("prehooked")
        layer.register_forward_pre_hook(save_output)       ## Input for the module will be grapped       
####################################################

dataiter = iter(trainloader)
images, labels = dataiter.next()
images = images.to(device)
out = model(images)


prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked


In [ ]:
# HW

#  1. Train with 4 bits for both weight and activation to achieve >90% accuracy
#  2. Find x_int and w_int for the 2nd convolution layer
#  3. Check the recovered psum has similar value to the un-quantized original psum
#     (such as example 1 in W3S2)

In [3]:
PATH = "result/resnet20_quant4bit/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 8987/10000 (90%)



In [ ]:
len(save_output.outputs[0][0])

In [34]:
w_bit = 4
weight_q = model.layer1[0].conv2.weight_q # quantized value is stored during the training
w_alpha = model.layer1[0].conv2.weight_quant.wgt_alpha
w_delta = w_alpha/(2**(w_bit-1)-1)
weight_int = weight_q/w_delta
print(weight_int) # you should see clean integer numbers

tensor([[[[ 0., -1., -1.],
          [-0., -2.,  3.],
          [-0., -2., -3.]],

         [[ 2., -2.,  2.],
          [-2., -2., -0.],
          [-0., -2., -3.]],

         [[-5., -1.,  7.],
          [ 1.,  1.,  7.],
          [-4.,  0.,  3.]],

         [[ 0., -2., -3.],
          [ 0., -3.,  0.],
          [ 0., -1., -1.]],

         [[-2., -2., -3.],
          [ 4., -7., -3.],
          [-0., -2., -3.]],

         [[ 1., -1., -2.],
          [-1.,  3., -2.],
          [ 3., -1., -1.]],

         [[ 1.,  1., -3.],
          [ 0., -5.,  6.],
          [ 1., -2., -0.]],

         [[ 2., -2., -1.],
          [ 1.,  4., -0.],
          [ 1., -0.,  1.]]],


        [[[-1.,  1.,  1.],
          [-1., -1., -1.],
          [-0., -3., -3.]],

         [[-1.,  2.,  1.],
          [ 0.,  1.,  5.],
          [-1.,  0., -0.]],

         [[-3.,  0., -2.],
          [-1.,  6., -0.],
          [-0.,  4.,  2.]],

         [[ 3.,  2.,  0.],
          [-2., -5., -5.],
          [ 0., -2., -0.]],

  

In [35]:
x_bit = 4
x = save_output.outputs[2][0]  # input of the 2nd conv layer
x_alpha  = model.layer1[0].conv2.act_alpha
x_delta = x_alpha/(2**x_bit-1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q/x_delta
print(x_int) # you should see clean integer numbers 

tensor([[[[ 1.,  0.,  0.,  ...,  3.,  2.,  1.],
          [ 0.,  1.,  1.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          ...,
          [ 8.,  8.,  9.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  2.,  2.,  1.],
          [ 1.,  2.,  2.,  ...,  2.,  2.,  1.]],

         [[ 0.,  0.,  0.,  ...,  3.,  3.,  2.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  1.],
          [ 0.,  0.,  0.,  ...,  3.,  1.,  2.],
          ...,
          [ 0.,  7.,  5.,  ...,  2.,  1.,  2.],
          [ 0.,  0.,  0.,  ...,  1.,  0.,  2.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

         [[ 4.,  0.,  1.,  ..., 10.,  2.,  2.],
          [ 1.,  0.,  2.,  ...,  8.,  2.,  2.],
          [ 0.,  0.,  3.,  ...,  9.,  2.,  2.],
          ...,
          [ 8.,  3.,  4.,  ...,  1.,  2.,  2.],
          [ 7.,  2.,  2.,  ...,  2.,  2.,  1.],
          [ 4.,  3.,  2.,  ...,  2.,  2.,  2.]],

         ...,

         [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  

In [50]:
conv_int = torch.nn.Conv2d(in_channels = 8, out_channels=8, kernel_size = 3, padding=1, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)
relu = nn.ReLU()
bn = nn.BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True).to(device)
output_int = conv_int(x_int)
output_int = output_int*w_delta*x_delta+save_output.outputs[1][0]
output_recovered = relu(output_int)


In [51]:
difference = abs(save_output.outputs[3][0] - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

tensor(3.4376e-07, device='cuda:0', grad_fn=<MeanBackward0>)


In [ ]:
#### input floating number / weight quantized version

conv_ref = torch.nn.Conv2d(in_channels = 16, out_channels=16, kernel_size = 3, bias = False)
conv_ref.weight = model.layer1[0].conv1.weight_q 

output_ref = conv_ref(x)
print(output_ref)

In [ ]:
#### input floating number / weight floating number version

conv_ref = torch.nn.Conv2d(in_channels = 16, out_channels=16, kernel_size = 3, bias = False)
weight = model.layer1[0].conv1.weight
mean = weight.data.mean()
std = weight.data.std()
conv_ref.weight = torch.nn.parameter.Parameter(weight.add(-mean).div(std))

output_ref = conv_ref(x)
print(output_ref)


In [ ]:
difference = abs( output_ref - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model